<a href="https://colab.research.google.com/github/inachenyx/SpeechSNN/blob/main/SpeakerExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torchaudio snntorch tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

### Speaker Recognition Pipeline

In [2]:
import torch
import torchaudio
import torchaudio.transforms as T
import snntorch as snn
from snntorch import utils
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchaudio.datasets import SPEECHCOMMANDS
import os
import random
import numpy as np
from tqdm import tqdm

# Set seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Dataset Setup

In [3]:
# Custom subclass to extract speaker labels
class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__(".", download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as f:
                return [os.path.join(self._path, line.strip()) for line in f]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]

train_dataset = SubsetSC("training")
test_dataset = SubsetSC("testing")


100%|██████████| 2.26G/2.26G [00:32<00:00, 75.1MB/s]


In [ ]:
mfcc_transform = T.MFCC(
    sample_rate=16000,
    n_mfcc=13,
    melkwargs={"n_fft": 400, "hop_length": 160, "n_mels": 40},
)

def extract_mfcc(dataset, max_len=80):
    features, labels, speakers = [], [], []
    speaker_to_idx = {}
    speaker_count = 0

    for waveform, sample_rate, _, speaker_id, _ in tqdm(dataset):
        mfcc = mfcc_transform(waveform).squeeze(0)  # [n_mfcc, time]
        mfcc = mfcc[:, :max_len]  # crop or pad
        if mfcc.shape[1] < max_len:
            pad = torch.zeros(mfcc.shape[0], max_len - mfcc.shape[1])
            mfcc = torch.cat([mfcc, pad], dim=1)

        if speaker_id not in speaker_to_idx:
            speaker_to_idx[speaker_id] = speaker_count
            speaker_count += 1

        features.append(mfcc)
        labels.append(speaker_to_idx[speaker_id])

    return torch.stack(features), torch.tensor(labels), speaker_to_idx

X_train, y_train, speaker_dict = extract_mfcc(train_dataset)
X_test, y_test, _ = extract_mfcc(test_dataset)
